# Tutorial 3: Setting up and managing phenopype projects

Phenopype projects are composed of a directory tree in which each folder contains the copy or a link to a single raw image file. Each raw should have only one folder, so that result or intermediate output from different iterations of processing and analysis are stored side by side. 


The project root folder should be separate from the raw data, e.g. as a folder inside of your project folder:
